## 数据处理部分，同Train.jupyter

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os,PIL,pathlib
import numpy as np
import pandas as pd
import warnings
from tensorflow import keras

warnings.filterwarnings("ignore")#忽略警告信息
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'


#映射
digit_map = {
    '零': 0,
    '一': 1,
    '二': 2,
    '三': 3,
    '四': 4,
    '五': 5,
    '六': 6,
    '七': 7,
    '八': 8,
    '九': 9,
    '十': 10,
    '百': 11,
    '千': 12,
    '万': 13,
    '亿': 14
}


train = pd.read_csv(os.path.join(os.getcwd(), "./data/train_dict.csv"))
test = pd.read_csv(os.path.join(os.getcwd(), "./data/dev_dict.csv"))
#训练数据的标签
train_image_label = [i for i in train["character"]]
test_image_label = [i for i in test["character"]]

#映射标签
train_label_ds = [digit_map[char] for char in train_image_label]
test_label_ds = [digit_map[char] for char in test_image_label]
# print(train_label_ds)

#将标签切片
train_label_ds = tf.data.Dataset.from_tensor_slices(train_label_ds)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_label_ds)
train_label_ds

#训练数据的具体路径
img_dir = os.path.join("./data/train/")
test_dir = os.path.join("./data/dev/")
train_image_paths = []
test_image_paths = []

for row in train.itertuples():
    suite_id = row[1]
    sample_id = row[2]
    code = row[3]
    train_image_paths.append(img_dir+"input"+"_"+str(suite_id)+"_"+str(sample_id)+"_"+str(code)+".jpg")
for row in test.itertuples():
    suite_id = row[1]
    sample_id = row[2]
    code = row[3]
    test_image_paths.append(test_dir+"input"+"_"+str(suite_id)+"_"+str(sample_id)+"_"+str(code)+".jpg")
    
#对图片路径进行切片
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)


height = 64
width = 64
batch_size = 128
epochs = 50

#图片预处理
def preprocess_image(image):
    image = tf.image.decode_jpeg(image,channels = 3)
    image = tf.image.resize(image,[height,width])
    return image / 255.0

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

#根据路径读取图片并进行预处理
train_image_ds = train_path_ds.map(load_and_preprocess_image,num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_image_ds = test_path_ds.map(load_and_preprocess_image,num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_ds = tf.data.Dataset.zip((train_image_ds,train_label_ds))
test_ds = tf.data.Dataset.zip((test_image_ds,test_label_ds))

train_ds = train_ds.batch(batch_size)#设置batch_size
train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.batch(batch_size)
test_ds = test_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


# 导入模型

In [2]:
from tensorflow.keras.models import load_model

# 导入保存的模型
model_ResNet = load_model('./model/model_ResNet18.keras')
model_CNN = load_model('./model/model_CNN.keras')

In [3]:
def printAccuracy(model):
    predictions = model.predict(test_ds)
    predicted_labels = tf.argmax(predictions, axis=1)
    total_samples = 0
    correct_predictions = 0

    test_label = test_label_ds.map(lambda x: tf.cast(x, tf.int64))
    predicted_labels = tf.cast(predicted_labels, tf.int64)

    for predicted_label, true_label in zip(predicted_labels, test_label):
        total_samples += 1
        if predicted_label == true_label:
            correct_predictions += 1

    accuracy = correct_predictions / total_samples
    print(accuracy)

In [4]:
printAccuracy(model_ResNet)

24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step
0.9946666666666667


In [5]:
printAccuracy(model_CNN)

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
0.9806666666666667
